# Unit 5 - Financial Planning

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets

my_btc = 1.2

my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD&format=json"

eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD&format=json"

In [ ]:
# Fetch current BTC price

btc_data = requests.get(btc_url).json()
btc_price = btc_data['data']['1']['quotes']['USD']['price']

# Fetch current ETH price
eth_data = requests.get(eth_url).json()
eth_price = eth_data['data']['1027']['quotes']['USD']['price']

# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("api_key")
alpaca_secret_key = os.getenv("api_secret_key")
base_url = os.getenv("base_url")
                           
# Create the Alpaca API object
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url, api_version = "v2")

In [ ]:
# Format current date as ISO format
current_date = pd.Timestamp("2024-01-05", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get todays closing prices for SPY and AGG
current_closing = api.get_bars(tickers, timeframe, start=current_date, end=current_date).df

# Reorganize the DataFrame
reorganize_current  = current_closing[['open', 'high', 'low','close', 'volume', 'symbol']]

# Separate ticker data
agg = reorganize_current[reorganize_current["symbol"] == "AGG"]
spy = reorganize_current[reorganize_current["symbol"] == "SPY"]

# Concatenate the ticker DataFrames
combined_closing = pd.concat([agg,spy], axis = 1, keys=["AGG", "SPY"] )


# Preview DataFrame
combined_closing.head()

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = combined_closing["AGG"]["close"].values[0]
spy_close_price = combined_closing["SPY"]["close"].values[0]

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price:0.2f}")
print(f"Current SPY closing price: ${spy_close_price:0.2f}")

In [ ]:
# Compute the current value of shares

my_spy_value = spy_close_price * my_spy

my_agg_value = agg_close_price * my_agg

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
crypto_amount = my_btc_value + my_eth_value
shares_amount = my_spy_value + my_eth_value

# Create savings DataFrame
df_savings = pd.DataFrame({"Amount": [crypto_amount, shares_amount]}, index=["crypto", "shares"])

# Display savings DataFrame
display(df_savings)


In [ ]:
# Plot savings pie chart
df_savings.plot.pie(y="Amount", title="savings pie chart",)

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
def savings_health(emergency_fund):
    while True:
        total_savings = df_savings['Amount'].sum()

        if total_savings >= emergency_fund:
            print(f"Congratulations! You have enough money in this fund: ${total_savings:0.2f}")
            break

        elif total_savings == emergency_fund:
            print(f"Congratulations! You have reached your financial goal: ${total_savings:0.2f}")
            break

        else:
            difference = emergency_fund - total_savings
            print(f"You are making progress! Your total savings are: ${total_savings:0.2f}")
            print(f"You need {difference} more to reach your financial goal.")



# Validate saving health
savings_health(emergency_fund)

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2019-01-05', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2024-01-05', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
historical_data = api.get_bars(tickers, timeframe, start=start_date, end=end_date).df

# Clean up data(Drop null values if any)
historical_data.dropna(inplace=True)

# Reorganize the DataFrame
reorganize_historical  = historical_data[['open', 'high', 'low','close', 'volume', 'symbol']]

# Separate ticker data
historical_agg = reorganize_historical[reorganize_historical["symbol"] == "AGG"]
historical_spy = reorganize_historical[reorganize_historical["symbol"] == "SPY"]

# Concatenate the ticker DataFrames
combined_historical_data = pd.concat([historical_agg,historical_spy], axis=1, keys=["AGG", "SPY"] )


# Display sample data
combined_historical_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
mc_thirty_years = MCSimulation(
    portfolio_data = combined_historical_data,
    weights=[.40,.60],
    num_simulation=500,
    num_trading_days=252*30,
)

In [ ]:
# Printing the simulation input data
mc_thirty_years.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
mc_thirty_years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
sim_plot_thirty = mc_thirty_years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_thirty = mc_thirty_years.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_statistics = mc_thirty_years.summarize_cumulative_return()

# Print summary statistics
print(summary_statistics)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(summary_statistics[8]*initial_investment,2)
ci_upper = round(summary_statistics[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(summary_statistics[8]*initial_investment,2)
ci_upper = round(summary_statistics[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
mc_five_years = MCSimulation(
    portfolio_data = combined_historical_data,
    weights=[.40,.60],
    num_simulation=500,
    num_trading_days=252*5,
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
mc_five_years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
sim_plot_five = mc_five_years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_five = mc_five_years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_statistics_five = mc_five_years.summarize_cumulative_return()

# Print summary statistics
print(summary_statistics_five)

In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(summary_statistics_five[8]*initial_investment,2)
ci_upper_five = round(summary_statistics_five[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
mc_ten_years = MCSimulation(
    portfolio_data = combined_historical_data,
    weights=[.40,.60],
    num_simulation=500,
    num_trading_days=252*10,
)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
mc_ten_years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
sim_plot_ten = mc_ten_years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_ten = mc_ten_years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_statistics_ten = mc_ten_years.summarize_cumulative_return()

# Print summary statistics
print(summary_statistics_ten)

In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(summary_statistics_ten[8]*initial_investment,2)
ci_upper_ten = round(summary_statistics_ten[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")

In [ ]:
print(f"The end, :)" )